```{=typst}
#set text(
  font: ("Times New Roman", "LXGW WenKai"),
  size: 11pt,
)

#image("Chapter6_1_1.jpg")

```


In [ ]:
import numpy as np
import scipy.optimize as opt

In [ ]:
# 导入数据
C0 = 0.8  # mg/L 上游污水浓度
C11 = 100  # mg/L 工厂 1 的排放浓度
C21 = 60  # mg/L 工厂 2 的排放浓度
C31 = 50  # mg/L 工厂 3 的排放浓度
V0 = 1000  # 10^12 L/min 上游水流量
V = 5  # 10^12 L/min 工厂的排放流量
b = 1  # 万元/((10^12 L/min) * (mg/L)) 处理系数
a12 = 0.9  # 工厂 1 和工厂 2 之间的自净系数
a23 = 0.6  # 工厂 2 和工厂 3 之间的自净系数

In [ ]:
Cn1 = np.array([C11, C21, C31])


def Cn(Cn2):
  if Cn2.size != 3:
    raise ValueError('Cn2 must be a 3-element array')

  C12, C22, C32 = Cn2

  C1 = (V0 * C0 + V * C12) / (V0 + V)
  C2 = ((V0 + V) * C1 * a12 + V * C22) / (V0 + V * 2)
  C3 = ((V0 + V * 2) * C2 * a23 + V * C32) / (V0 + V * 3)

  return np.array([C1, C2, C3])


def Z(Cn2):
  if Cn2.size != 3:
    raise ValueError('Cn2 must be a 3-element array')

  return -V * b * (sum(Cn2) - C11 - C21 - C31)


nlc0 = opt.NonlinearConstraint(lambda X: X, np.zeros(3), Cn1)
nlc1 = opt.NonlinearConstraint(Cn, np.zeros(3), np.ones(3))
nlc2 = opt.NonlinearConstraint(Z, 0, np.inf)

result = opt.minimize(Z, Cn1, constraints=[nlc0, nlc1, nlc2])


_1X = result.x
print(f'最优解为：{_1X}')
print(f'最优值为：{result.fun}')

此时最少花费 489.5 万元。


In [ ]:
def Cp(Cn2):
  C1, C2 = Cn(Cn2)[0:2]
  return np.array([C0, C1 * a12, C2 * a23])


nlc3 = opt.NonlinearConstraint(Cp, np.zeros(3), np.ones(3))

result = opt.minimize(Z, Cn1, constraints=[nlc0, nlc2, nlc3])

_2X = result.x
print(f'最优解为：{_2X}')
print(f'最优值为：{result.fun}')

此时最少花费 183.3 万元。
